In [1]:
## 載入預先處理好的資料
import dill
with open('model.pkl', 'rb') as in_strm:
    d = dill.load(in_strm)

In [2]:
## EF666.txt 放在thor 阿芳的根目錄下
EF_sent = [i.replace('\n','')for i in open('../../EF666.txt').readlines()]

In [3]:
## get have VT_tag sentence
Tmp_VT = []
for i in EF_sent:
    if 'VT+' in i or 'VT-' in i:
        Tmp_VT.append(i)

In [58]:
from tool import find_pattern_spacy_result , pair_key , diff2before_after_taglook
from rule import match 

In [99]:
def look_index_result(index):
    result , tag_sen = find_pattern_spacy_result(Tmp_VT[index])
    for item in match(result , tag_sen):
        rule , pair = item[0] , item[1]
        key = pair_key(pair)
        print('Rule : ',rule ,"|", 'Main verb : ',key ,"|", pair )  
    print("# Sentence : #")
    print('##################################################')
    print(tag_sen)
    print('##################################################')
    
    
def result_past(index):
    rule = ['yesterday','last year']
    tag_sen , tag = diff2before_after_taglook(Tmp_VT[index],'VT')
    for i in rule:
        if i in tag_sen:
            return tag_sen.replace('\u3000',' ')
        
def result_now(index):
    rule = ['now']
    tag_sen , tag = diff2before_after_taglook(Tmp_VT[index],'VT')
    for i in rule:
        if i in tag_sen:
            return tag_sen.replace('\u3000',' ')
        
def result_future(index):
    rule = ['tomorrow', 'next year']
    tag_sen , tag = diff2before_after_taglook(Tmp_VT[index],'VT')
    for i in rule:
        if i in tag_sen:
            return tag_sen.replace('\u3000',' ')
        
def count_now(d , rule , verb):
    sen = []
    for i in d[rule][verb]:
        if result_now(i):
            sen.append(result_now(i))
    return sen

def count_future(d , rule , verb):
    sen = []
    for i in d[rule][verb]:
        if result_future(i):
            sen.append(result_future(i))
    return sen
    
    
def count_past(d , rule , verb):
    sen = []
    for i in d[rule][verb]:
        if result_past(i):
            sen.append(result_past(i))
    return sen
    
    
def look_rule_count(d , rule):
    count = 0
    for verb in d[rule].keys():
        count+=len(d[rule][verb])
    return count
    
def look_all_rule_count(d):
    for rule in d.keys():
        print(rule , look_rule_count(d , rule))
    return

def look_rule_verb_count(d , rule , number=None):
    dd = {}
    for verb in d[rule].keys():
        count = 0
        count+=len(d[rule][verb])
        dd[verb] = count
    print(sorted(dd.items() , key = lambda x:x[1] , reverse=True)[:number])
    return 

def confuse_look(d , rule , verb , number=10):
    print("# Rule #")
    for i in d[rule][verb][:number]:
        look_index_result(i)
        print('-------------------------END--------------------------------------------')
    print('')
    print('')

In [6]:
#     20種組合      #
# 'simple->nowing'
# 'now->now'
# 'simple->simple'
# 'now->future'
# 'future->now'
# 'finish->finish'
# 'nowing->nowing'
# 'future->future'
# 'simple->finish'
# 'finishing->finish'
# 'nowing->simple'
# 'simple->finishing'
# 'nowing->finishing'
# 'finishing->finishing'
# 'finish->finishing'
# 'finish->simple'
# 'nowing->finish'
# 'finishing->simple'
# 'finishing->nowing'
# 'finish->nowing'
# 'now->past'

In [7]:
## 20種組合，各個個數
#look_all_rule_count(d)

In [12]:
## 單個規則個數
look_rule_count(d , 'now->past')

39527

In [13]:
## 查看單個規則所擁有的verb的個數(已經sorted過!)，number參數是一次要顯示幾筆。
look_rule_verb_count(d ,'now->past' , number=10)

[('be', 5840), ('have', 2982), ('get', 1005), ('make', 924), ('work', 880), ('go', 814), ('do', 730), ('like', 659), ('meet', 643), ('want', 632)]


In [14]:
## 學姊對各數有懷疑或是主要的verb有疑問，請用confuse_look()，ex: be怎麼會有2501?????
look_rule_verb_count(d , 'now->past' , 10)

[('be', 5840), ('have', 2982), ('get', 1005), ('make', 924), ('work', 880), ('go', 814), ('do', 730), ('like', 659), ('meet', 643), ('want', 632)]


In [15]:
## number=3意思是取2501中的前3筆
## confuse_look給你ngram之後含有verb的spacy_tag及這個tag集合應該配對到哪一個rule，及原始的句子。
confuse_look(d , 'now->past' , 'have' , number=3)

# Rule #
Rule :  now->past | Main verb :  have | ([('have', 'VBP')], [('had', 'VBD')])
Rule :  simple->simple | Main verb :  have | ([('have', 'VBP')], [('had', 'VBD')])
# Sentence : #
##################################################
I [-have//VT-]{+had//VT+} a fever , headache and cough .
##################################################
-------------------------END--------------------------------------------
Rule :  now->past | Main verb :  have | ([('have', 'VBP')], [('had', 'VBD')])
Rule :  simple->simple | Main verb :  have | ([('have', 'VBP')], [('had', 'VBD')])
# Sentence : #
##################################################
Hi , I [-have//VT-]{+had//VT+} a cold on the day you married sorry everybody , the doctor said I should n't go out because its dangerous for me , because the fever is very high he said I should stay in the house , so I relaxed in the bed .
##################################################
-------------------------END-------------------------------------

In [94]:
look_rule_count(d , 'now->future')

6665

In [91]:
look_rule_verb_count(d ,'now->future' , number=10)

[('be', 1527), ('have', 544), ('go', 401), ('start', 250), ('see', 208), ('make', 166), ('get', 149), ('take', 144), ('come', 131), ('bre', 118)]


In [100]:
count_future(d , 'now->future' , 'have')

['Jones [-has//VT-]{+will have//VT+} lunch with Margaret Kendall tomorrow , Tuesday 12th , at La Maison restaurant .',
 'Dear Dad , Wanted to tell you that next year I {+will//VT+} have done my course in Design and I {+will//VT+} marry Igor .',
 'tomorrow night we [-are//VT-]{+will have//VT+} one classical music concert .',
 'I hope you [-have//VT-]{+will have//VT+} a great time tomorrow and I really look forward to meeting Lucy some time next week .',
 'Jones tomorrow , Tuesday the 12th at 2 pm you {+will//VT+} have lunch with Margaret Kendall in La Maison restaurant Taken by : Marina',
 'I think we {+will//VT+} have to buy furniture tomorrow . That is good news . The bad news is , that Mr KHITO will leave our company next week . He told Bob that it is due to his bad health .',
 'Since next year we are going to stay in Nagasaki so we {+will//VT+} have to pay mortgages , utilities and so on .',
 "I hope that next year I [-have//VT-]{+will have//VT+} more confidence to speak , because I

In [101]:
print(len(count_future(d , 'now->future' , 'have')))

8
